<a href="https://colab.research.google.com/github/SURE18/Practical-Machine-Learning-with-Tensorflow/blob/master/Assignment_4_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Setup**

Install and import all the necessary libraries for the assignment.

In [77]:
!pip install tensorflow==2.0.0-rc0

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston
import tensorflow as tf
import pandas as pd

tf.random.set_seed(1)

### **Importing the dataset**

In [0]:
boston_dataset = load_boston()

data_X = pd.DataFrame(boston_dataset.data, columns=boston_dataset.feature_names)
data_Y = pd.DataFrame(boston_dataset.target, columns=["target"])
data = pd.concat([data_X, data_Y], axis=1)

In [79]:
train, test = train_test_split(data, test_size=0.2, random_state=1)
train, val = train_test_split(train, test_size=0.2, random_state=1)
print(len(train), "train examples")
print(len(val), "validation examples")
print(len(test), "test examples")

323 train examples
81 validation examples
102 test examples


In [104]:
train.describe().T

,count,mean,std,min,25%,50%,75%,max
CRIM,323.0,3.882044,9.680410,0.00632,0.082865,0.25387,3.944055,88.9762
ZN,323.0,12.233746,24.323780,0.00000,0.000000,0.00000,19.000000,100.0000
INDUS,323.0,11.121796,6.876617,0.46000,5.190000,8.56000,18.100000,27.7400
CHAS,323.0,0.080495,0.272481,0.00000,0.000000,0.00000,0.000000,1.0000
NOX,323.0,0.553683,0.118625,0.38500,0.447500,0.53200,0.624000,0.8710
RM,323.0,6.276935,0.691051,3.56100,5.886500,6.19300,6.666500,8.7800
AGE,323.0,68.279876,28.452257,2.90000,43.550000,77.70000,93.900000,100.0000
DIS,323.0,3.864749,2.154972,1.12960,2.120350,3.36030,5.237900,12.1265
RAD,323.0,9.721362,8.801984,1.00000,4.000000,5.00000,24.000000,24.0000
TAX,323.0,406.588235,170.365557,187.00000,279.000000,329.00000,666.000000,711.0000


Converting the Pandas DataFrames into Tensorflow Datasets

In [0]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [0]:
batch_size = 32
train_ds = df_to_dataset(train, True, batch_size)
val_ds = df_to_dataset(val, False, batch_size)
test_ds = df_to_dataset(test, False, batch_size)

In [90]:
for feature_batch, label_batch in train_ds.take(1):
  print('Every feature:',list(feature_batch.keys()))
  print('A batch of CRIMs:', feature_batch)
  print('A batch of targets:', label_batch)

Every feature: ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']
A batch of CRIMs: {'CRIM': <tf.Tensor: id=1596, shape=(32,), dtype=float64, numpy=
array([ 4.75237,  0.17171,  0.07875,  0.66351,  0.06129,  0.49298,
        0.35233,  4.54192,  0.95577,  9.39063, 14.4383 ,  6.80117,
        4.87141, 15.5757 ,  0.25387,  0.09378,  0.0351 ,  0.1265 ,
        0.24103,  0.01965,  0.22969,  7.75223,  0.1146 , 14.4208 ,
        0.7857 ,  0.09604,  0.33045,  0.52058, 13.9134 ,  1.41385,
        0.15936, 14.3337 ])>, 'ZN': <tf.Tensor: id=1605, shape=(32,), dtype=float64, numpy=
array([ 0. , 25. , 45. , 20. , 20. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        0. ,  0. ,  0. ,  0. , 12.5, 95. , 25. ,  0. , 80. ,  0. ,  0. ,
       20. ,  0. , 20. , 40. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ])>, 'INDUS': <tf.Tensor: id=1598, shape=(32,), dtype=float64, numpy=
array([18.1 ,  5.13,  3.44,  3.97,  3.33,  9.9 , 21.89, 18.1 ,  8.14,
       18.1 , 18.1 , 18.1 

In [112]:
for key in feature_batch.keys():
  print(type(key) ,' ',feature_batch[key][0:3])

<class 'str'>   tf.Tensor([4.75237 0.17171 0.07875], shape=(3,), dtype=float64)
<class 'str'>   tf.Tensor([ 0. 25. 45.], shape=(3,), dtype=float64)
<class 'str'>   tf.Tensor([18.1   5.13  3.44], shape=(3,), dtype=float64)
<class 'str'>   tf.Tensor([0. 0. 0.], shape=(3,), dtype=float64)
<class 'str'>   tf.Tensor([0.713 0.453 0.437], shape=(3,), dtype=float64)
<class 'str'>   tf.Tensor([6.525 5.966 6.782], shape=(3,), dtype=float64)
<class 'str'>   tf.Tensor([86.5 93.4 41.1], shape=(3,), dtype=float64)
<class 'str'>   tf.Tensor([2.4358 6.8185 3.7886], shape=(3,), dtype=float64)
<class 'str'>   tf.Tensor([24.  8.  5.], shape=(3,), dtype=float64)
<class 'str'>   tf.Tensor([666. 284. 398.], shape=(3,), dtype=float64)
<class 'str'>   tf.Tensor([20.2 19.7 15.2], shape=(3,), dtype=float64)
<class 'str'>   tf.Tensor([ 50.92 378.08 393.87], shape=(3,), dtype=float64)
<class 'str'>   tf.Tensor([18.13 14.44  6.68], shape=(3,), dtype=float64)


### Defining Feature Columns

In [0]:
# define feature_columns as a list of features using functions from tf.feature_column

feature_columns = []
for key in feature_batch.keys():
  feature_columns.append(tf.feature_column.numeric_column(key))


### Building the model

In [0]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

Model should contain following layers:

```
feature_layer

Dense(1, activation=None)
```

Use 'Adam' optimizer

Use 'mse' as your loss and metric

In [0]:
# Build and compile your model in this cell.


In [0]:
model.fit(train_ds, validation_data=val_ds, epochs=200)

In [0]:
loss, mse = model.evaluate(test_ds)
print("Mean Squared Error - Test Data", mse)